In [1]:
# Necessary and residual imports
import nltk
from nltk.corpus import udhr
import keras
from nltk import FreqDist
from gensim.models.word2vec import Word2Vec
import pandas as pd
import regex
import nltk.tokenize.casual
import tensorflow as tf

In [2]:
import collections
import os
import pathlib
import re
import string
import sys
import tempfile
import time
import gc

import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Load dataset: (from https://huggingface.co/datasets/versae/bibles)
# https://stackoverflow.com/questions/39263929/how-can-i-read-tar-gz-file-using-pandas-read-csv-with-gzip-compression-option
df = pd.read_csv('books_labels.tar.gz', compression='gzip', header=0, sep=',', quotechar='"', on_bad_lines='skip')

In [4]:
# Select Spanish texts
# https://stackoverflow.com/questions/17424182/extracting-all-rows-from-pandas-dataframe-that-have-certain-value-in-a-specific
#spanish_df = df[df['language'] == 'SPA']['text']
#spanish_list = spanish_df.to_list() # https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://ioflood.com/blog/dataframe-to-list-pandas/%23:~:text%3DIt%2520is%2520utilized%2520with%2520the,tolist()%2520.%26text%3DIn%2520the%2520example%2520above%252C%2520we,convert%2520it%2520into%2520a%2520list.&ved=2ahUKEwiP3deMnLSFAxVD4ckDHUo7BsEQFnoECA4QAw&usg=AOvVaw3GgfIVdsVo9Dxul02uata1

In [5]:
# Select Spanish texts
# https://stackoverflow.com/questions/17424182/extracting-all-rows-from-pandas-dataframe-that-have-certain-value-in-a-specific
spanish_df = df[df['language'] == 'SPA']
portuguese_df = df[df['language'] == 'POR']

spanish_df = spanish_df.sort_values(by=['file_name_translation', 'id'])[df['file_name_translation'] == 'Nueva Biblia de las Américas (NBLA)'] # https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.reddit.com/r/Reformed/comments/xoye82/most_accurate_spanish_bible_translations_spanish/&ved=2ahUKEwj7qbSKtbuFAxV4IEQIHbm6B_UQFnoECCwQAQ&usg=AOvVaw2YZJ8eHalVoloIoaaATROH
spanish_df = spanish_df.set_index('id')
spanish_df = spanish_df.drop(columns=['books_labels.csv', 'codebook', 'language', 'book_file_name', 'file_name_translation', 'source', 'year', 'genre', 'genre-multilabel', 'testament', 'division'])
#spanish_list = spanish_df['text'].to_list() # https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://ioflood.com/blog/dataframe-to-list-pandas/%23:~:text%3DIt%2520is%2520utilized%2520with%2520the,tolist()%2520.%26text%3DIn%2520the%2520example%2520above%252C%2520we,convert%2520it%2520into%2520a%2520list.&ved=2ahUKEwiP3deMnLSFAxVD4ckDHUo7BsEQFnoECA4QAw&usg=AOvVaw3GgfIVdsVo9Dxul02uata1

/var/folders/05/btfxjzv52_369qlngpc4v8fc0000gn/T/ipykernel_13259/44389737.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  spanish_df = spanish_df.sort_values(by=['file_name_translation', 'id'])[df['file_name_translation'] == 'Nueva Biblia de las Américas (NBLA)'] # https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.reddit.com/r/Reformed/comments/xoye82/most_accurate_spanish_bible_translations_spanish/&ved=2ahUKEwj7qbSKtbuFAxV4IEQIHbm6B_UQFnoECCwQAQ&usg=AOvVaw2YZJ8eHalVoloIoaaATROH


In [6]:
portuguese_df = portuguese_df.sort_values(by=['file_name_translation', 'id'])[df['file_name_translation'] == 'SF_2009-01-20_POR_ACF_(PORTUGUESE CORRIGIDA FIEL (1753_1995))']
portuguese_df = portuguese_df.set_index('id')
portuguese_df = portuguese_df.drop(columns=['books_labels.csv', 'codebook', 'language', 'book_file_name', 'file_name_translation', 'source', 'year', 'genre', 'genre-multilabel', 'testament', 'division'])

#portuguese_list = portuguese_df['text'].to_list()

/var/folders/05/btfxjzv52_369qlngpc4v8fc0000gn/T/ipykernel_13259/2818949468.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  portuguese_df = portuguese_df.sort_values(by=['file_name_translation', 'id'])[df['file_name_translation'] == 'SF_2009-01-20_POR_ACF_(PORTUGUESE CORRIGIDA FIEL (1753_1995))']


In [7]:
# Clean, format, and tokenize Spanish texts
#spanish_list = [regex.sub(r'\([a-zA-z0-9]\)', '', item) for item in spanish_list]
# spanish_string = ''.join(spanish_list_clean)
# spanish_list_clean = spanish_string.split('.')
# spanish_list_clean = [item.strip(' ') for item in spanish_list_clean]

In [8]:
spanish_df.head(5)

,text
id,
b.1CH.001.001,"(A)Adán, Set, Enós,"
b.1CH.001.002,"Cainán, Mahalaleel, Jared,"
b.1CH.001.003,"Enoc, Matusalén, Lamec,"
b.1CH.001.004,"Noé, Sem, Cam y Jafet."
b.1CH.001.005,"(B)Los hijos de Jafet fueron Gomer, Magog, Mad..."


In [9]:
spanish_df = spanish_df.rename(columns={'text':'spanish_text'})
portuguese_df = portuguese_df.rename(columns={'text':'portuguese_text'})

multi_df = spanish_df.join(portuguese_df)
multi_df = multi_df.dropna()

print("Spanish:", multi_df['spanish_text']['b.1CH.001.011'],"Portuguese:", multi_df['portuguese_text']['b.1CH.001.011'])

spanish_list = multi_df['spanish_text'].to_list()
portuguese_list = multi_df['portuguese_text'].to_list()

print('Spanish:', spanish_list[11], 'Portuguese:', portuguese_list[11])

Spanish: (C)Y Mizrayim fue el padre del pueblo de Ludim, Anamim, Lehabim, Neftuhim, Portuguese: E Mizraim gerou aos ludeus e aos anameus e aos leabeus e aos naftueus,
Spanish: Canaán fue el padre de Sidón su primogénito, y de Het, Portuguese: E Canaã gerou a Sidom seu primogênito, e a Hete,


In [10]:
# Format and tokenize lists to make corpi
spanish_list = [regex.sub(r'\([a-zA-z0-9]\)', '', item) for item in spanish_list]
spanish_corpus = [nltk.tokenize.casual_tokenize(item) for item in spanish_list]
print(spanish_corpus[0])
portuguese_corpus = [nltk.tokenize.casual_tokenize(item) for item in portuguese_list]
print(portuguese_corpus[0])

['Adán', ',', 'Set', ',', 'Enós', ',']
['ADÃO', ',', 'Sete', ',', 'Enos', ',']


In [11]:
# Select Portuguese texts
# https://stackoverflow.com/questions/17424182/extracting-all-rows-from-pandas-dataframe-that-have-certain-value-in-a-specific
# portuguese_df = df[df['language'] == 'POR']['text']
# portuguese_list = portuguese_df.to_list()

In [12]:
# Garbage collect old variables
del df
del spanish_list, portuguese_list, spanish_df, portuguese_df, multi_df
gc.collect()

0

In [13]:
# Prepare vocabulary
spanish_encoder_texts, spanish_input_texts, spanish_target_texts = [], [], []
portuguese_encoder_texts, portuguese_input_texts, portuguese_target_texts = [], [], []
spanish_vocabulary = set()
portuguese_vocabulary = set()
spanish_start_token = '[SPSTART]'
portuguese_start_token = '[POSTART]'
stop_token = '[END]'
unknown_token = '[UNK]'
pad_token = '[PAD]'
spanish_vocabulary.add(spanish_start_token)
spanish_vocabulary.add(stop_token)
spanish_vocabulary.add(unknown_token)
spanish_vocabulary.add(pad_token)
portuguese_vocabulary.add(portuguese_start_token)
portuguese_vocabulary.add(stop_token)
portuguese_vocabulary.add(unknown_token)
portuguese_vocabulary.add(pad_token)

for spanish_text in spanish_corpus:
    spanish_encoder_texts.append([spanish_start_token] + spanish_text + [stop_token])
    spanish_input_texts.append([spanish_start_token] + spanish_text)
    spanish_target_texts.append(spanish_text + [stop_token])
    for char in spanish_text:
        if char not in spanish_vocabulary:
            spanish_vocabulary.add(char)

for portuguese_text in portuguese_corpus:
    portuguese_encoder_texts.append([portuguese_start_token] + portuguese_text + [stop_token])
    portuguese_input_texts.append([portuguese_start_token] + portuguese_text)
    portuguese_target_texts.append(portuguese_text + [stop_token])
    for char in portuguese_text:
        if char not in portuguese_vocabulary:
            portuguese_vocabulary.add(char)

unified_vocabulary = spanish_vocabulary.union(portuguese_vocabulary)

print(len(spanish_vocabulary), len(unified_vocabulary), len(portuguese_vocabulary))
print(spanish_encoder_texts[0], portuguese_input_texts[0], portuguese_target_texts[0])

26936 50891 29203
['[SPSTART]', 'Adán', ',', 'Set', ',', 'Enós', ',', '[END]'] ['[POSTART]', 'ADÃO', ',', 'Sete', ',', 'Enos', ','] ['ADÃO', ',', 'Sete', ',', 'Enos', ',', '[END]']


In [14]:
# Finish vocabulary
spanish_vocabulary = sorted(spanish_vocabulary)
portuguese_vocabulary = sorted(portuguese_vocabulary)

# Define maxima
spanish_vocab_size = len(spanish_vocabulary)
portuguese_vocab_size = len(portuguese_vocabulary)
unified_vocab_size = len(unified_vocabulary)
max_spanish_seq_length = max([len(txt) for txt in spanish_target_texts])
max_portuguese_seq_length = max([len(txt) for txt in portuguese_target_texts])
max_unified_seq_length = max(max_spanish_seq_length, max_portuguese_seq_length)

# Create indicies
spanish_token_index = dict([(token, i) for i, token in
                          enumerate(spanish_vocabulary)])
portuguese_token_index = dict([(token, i) for i, token in
                          enumerate(portuguese_vocabulary)])
unified_token_index = dict([(token, i) for i, token in
                          enumerate(unified_vocabulary)])
reverse_spanish_token_index = dict([(i, token) for token, i in
                          spanish_token_index.items()])
reverse_portuguese_token_index = dict([(i, token) for token, i in
                          portuguese_token_index.items()])
reverse_unified_token_index = dict([(i, token) for token, i in
                          unified_token_index.items()])
print(max_spanish_seq_length, max_unified_seq_length, max_portuguese_seq_length)

114 114 109


In [15]:
def convert_text_to_indices(texts, token_index, max_seq_length, pad_token='[PAD]'):
    data = np.zeros((len(texts), max_seq_length),
                    dtype='int32')
    for i, text in enumerate(texts):
        for t, token in enumerate(text):
            data[i, t] = token_index[token]
        for t in range(len(text), max_seq_length):
            data[i, t] = token_index[pad_token]
    
    return data

In [16]:
import numpy as np

# Convert sentences to numpy arrays
spanish_encoder_input_data = convert_text_to_indices(spanish_encoder_texts, unified_token_index, max_spanish_seq_length+1)
spanish_decoder_input_data = convert_text_to_indices(spanish_input_texts, unified_token_index, max_spanish_seq_length)
spanish_decoder_target_data = convert_text_to_indices(spanish_target_texts, unified_token_index, max_spanish_seq_length)

portuguese_encoder_input_data = convert_text_to_indices(portuguese_encoder_texts, unified_token_index, max_portuguese_seq_length+1)
portuguese_decoder_input_data = convert_text_to_indices(portuguese_input_texts, unified_token_index, max_portuguese_seq_length)
portuguese_decoder_target_data = convert_text_to_indices(portuguese_target_texts, unified_token_index, max_portuguese_seq_length)

# spanish_encoder_input_data = np.zeros((len(spanish_encoder_texts), max_spanish_seq_length),
#                                dtype='int32')
# spanish_decoder_input_data = np.zeros((len(spanish_input_texts), max_spanish_seq_length),
#                                dtype='int32')
# spanish_decoder_target_data = np.zeros((len(spanish_target_texts), max_spanish_seq_length),
#                                dtype='int32')

# portuguese_encoder_input_data = np.zeros((len(portuguese_encoder_texts), max_portuguese_seq_length),
#                                dtype='int32')
# portuguese_decoder_input_data = np.zeros((len(portuguese_input_texts), max_portuguese_seq_length),
#                                dtype='int32')
# portuguese_decoder_target_data = np.zeros((len(portuguese_target_texts), max_portuguese_seq_length),
#                                dtype='int32')



# for i, (input_text, target_text) in enumerate(zip(spanish_input_texts, spanish_target_texts)):
#     for t, token in enumerate(input_text):
#         spanish_encoder_input_data[
#             i, t] = unified_token_index[token]
#     for t, token in enumerate(target_text):
#         spanish_decoder_input_data[
#             i, t] = unified_token_index[token]
#         if t > 0:
#             spanish_decoder_target_data[i, t - 1] = spanish_token_index[token]

# for i, (input_text, target_text) in enumerate(zip(portuguese_input_texts, portuguese_target_texts)):
#     for t, token in enumerate(input_text):
#         portuguese_encoder_input_data[
#             i, t] = unified_token_index[token]
#     for t, token in enumerate(target_text):
#         portuguese_decoder_input_data[
#             i, t] = unified_token_index[token]
#         if t > 0:
#             portuguese_decoder_target_data[i, t - 1] = portuguese_token_index[token]

print([reverse_unified_token_index[value] for value in portuguese_encoder_input_data[0]])
print([reverse_unified_token_index[value] for value in portuguese_decoder_input_data[0]])
print([reverse_unified_token_index[value] for value in portuguese_decoder_target_data[0]])
print([reverse_unified_token_index[value] for value in spanish_encoder_input_data[0]])
print([reverse_unified_token_index[value] for value in spanish_decoder_input_data[0]])
print([reverse_unified_token_index[value] for value in spanish_decoder_target_data[0]])

['[POSTART]', 'ADÃO', ',', 'Sete', ',', 'Enos', ',', '[END]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
['[POSTART]', 'ADÃO'

In [17]:
# Display some input
spanish_encoder_input_data

array([[30283, 18216, 17907, ..., 40616, 40616, 40616],
       [30283,  9217, 17907, ..., 40616, 40616, 40616],
       [30283,  5487, 17907, ..., 40616, 40616, 40616],
       ...,
       [30283,  9154, 16042, ..., 40616, 40616, 40616],
       [30283, 47221, 30766, ..., 40616, 40616, 40616],
       [30283, 25911, 39241, ..., 40616, 40616, 40616]], dtype=int32)

In [18]:
# Define hyperparameters
batch_size = 32
epochs = 20
num_neurons = 256

In [19]:
num_layers = 2
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1

In [20]:
# Import from customized file
from transformers import *

In [21]:
# Define the encoder and two decoders
# encoder = Encoder(
#     num_layers=num_layers,
#     d_model=d_model,
#     num_heads=num_heads,
#     dff=dff,
#     vocab_size=unified_vocab_size,
#     dropout_rate=dropout_rate)

# spanish_decoder = Decoder(
#     num_layers=num_layers,
#     d_model=d_model,
#     num_heads=num_heads,
#     dff=dff,
#     vocab_size=unified_vocab_size,
#     dropout_rate=dropout_rate)

# portuguese_decoder = Decoder(
#     num_layers=num_layers,
#     d_model=d_model,
#     num_heads=num_heads,
#     dff=dff,
#     vocab_size=unified_vocab_size,
#     dropout_rate=dropout_rate)

In [22]:
# Create the transformers from the encoder and decoder
transformer = Transformer(num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=unified_vocab_size,
    target_vocab_size=unified_vocab_size,
    dropout_rate=dropout_rate)
# spanish_transformer = ComposedTransformer(encoder, spanish_decoder, spanish_vocab_size)
# portuguese_transformer = ComposedTransformer(encoder, portuguese_decoder, portuguese_vocab_size)

2024-04-25 08:02:13.084479: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2024-04-25 08:02:13.084501: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-04-25 08:02:13.084511: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-04-25 08:02:13.084527: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-25 08:02:13.084539: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [23]:
# Define learning rate
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [24]:
# Compile both models
transformer.compile(
    loss=masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy])

In [25]:
# Prepare training data
# dataset = tf.data.Dataset.from_tensor_slices(((spanish_encoder_input_data[1000:2000], portuguese_decoder_input_data[1000:2000]), portuguese_decoder_target_data[1000:2000]))
# batched_dataset = dataset.batch(batch_size)
# spanish_dataset = tf.data.Dataset.from_tensor_slices(((spanish_encoder_input_data, spanish_decoder_input_data), spanish_decoder_target_data))
# portuguese_dataset = tf.data.Dataset.from_tensor_slices(((spanish_encoder_input_data, spanish_decoder_input_data), spanish_decoder_target_data))
# spanish_batched_dataset = spanish_dataset.batch(batch_size)
# portuguese_batched_dataset = portuguese_dataset.batch(batch_size)

In [26]:
# Give a summary of the transformer
transformer.summary()

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder (Encoder)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Decoder)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [27]:
# Define a translator
class Translator(tf.Module):
  def __init__(self, transformer, token_index, reverse_token_index, input_start_char, output_start_char):
    self.transformer = transformer
    self.token_index = token_index
    self.reverse_token_index = reverse_token_index
    self.input_start_char = input_start_char
    self.output_start_char = output_start_char

  def __call__(self, sentence, max_length):
    # The input sentence is Portuguese, hence adding the `[START]` and `[END]` tokens.
    # assert isinstance(sentence, tf.Tensor)
    # if len(sentence.shape) == 0:
    #   sentence = sentence[tf.newaxis]
    tokenized_sentence = [self.input_start_char] + nltk.tokenize.casual_tokenize(sentence.lower()) + ['[END]']
    sentence_tensor = np.zeros(len(tokenized_sentence), dtype='int64')
    for t, token in enumerate(tokenized_sentence):
      if token in self.token_index:
        sentence_tensor[t] = self.token_index[token]
      else:
        sentence_tensor[t] = self.token_index['[UNK]']
    #sentence = self.tokenizers.pt.tokenize(sentence).to_tensor()

    encoder_input = sentence_tensor[tf.newaxis]
    
    # As the output language is English, initialize the output with the
    # English `[START]` token.
    #start_end = self.tokenizers.en.tokenize([''])[0]
    start = self.token_index[self.output_start_char]
    end = self.token_index['[END]']

    # `tf.TensorArray` is required here (instead of a Python list), so that the
    # dynamic-loop can be traced by `tf.function`.
    output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
    output_array = output_array.write(0, start)
    for i in tf.range(max_length):
      output = tf.transpose(output_array.stack())[tf.newaxis]
      predictions = self.transformer([encoder_input, output], training=False)

      # Select the last token from the `seq_len` dimension.
      predictions = predictions[:, -1:, :]  # Shape `(batch_size, 1, vocab_size)`.

      predicted_id = tf.argmax(predictions, axis=-1)

      # Concatenate the `predicted_id` to the output which is given to the
      # decoder as its input.
      output_array = output_array.write(i+1, predicted_id[0][0])

      if predicted_id == end:
        break

    output = tf.transpose(output_array.stack())
    # The output shape is `(1, tokens)`. https://www.tensorflow.org/api_docs/python/tf/cast
    text = [self.reverse_token_index[item.numpy()] for item in output]  # Shape: `()`.

    # `tf.function` prevents us from using the attention_weights that were
    # calculated on the last iteration of the loop.
    # So, recalculate them outside the loop.
    # self.transformer([encoder_input, output[:,:-1]], training=False)
    # attention_weights = self.transformer.decoder.last_attn_scores

    return text

In [28]:
from nltk.translate import meteor_score
import random
def calculate_average_meteor(input_dataset, confirmation_dataset, translator, numtests, maxlen):
    score = 0
    i = 0
    while i < numtests:
        randindex = random.randrange(0, len(input_dataset))
        if len(input_dataset[randindex]) <= maxlen:
            expected = confirmation_dataset[randindex]
            result = translator(' '.join(input_dataset[randindex]), len(confirmation_dataset[randindex]))
            score += meteor_score.single_meteor_score(result[1:], expected)
            i += 1
    return score / i

In [29]:
# Train the transformers
tf.config.run_functions_eagerly(True)
# transformer.fit(batched_dataset, epochs=epochs)

In [30]:
print(len(spanish_encoder_input_data), len(portuguese_decoder_input_data), len(portuguese_decoder_target_data))
for t in range(7):
    print(f"Epoch {t}:")
    for i in range(1):
        dataset = tf.data.Dataset.from_tensor_slices(((spanish_encoder_input_data, portuguese_decoder_input_data), portuguese_decoder_target_data))
        dataset = dataset.shuffle(dataset.cardinality())
        batched_dataset = dataset.batch(batch_size)
        transformer.fit(batched_dataset)
    sentence = 'En el principio Dios creó los cielos y la tierra.'
    translator = Translator(transformer, unified_token_index, reverse_unified_token_index, '[SPSTART]', '[POSTART]')
    print(calculate_average_meteor(spanish_corpus, portuguese_corpus, translator, 10, 20))
    print(translator(sentence,20))
    # sentence = 'Estas son las palabras de Amós, que era un pastor de Tecoa'
    # translator = Translator(transformer, unified_token_index, reverse_unified_token_index, unified_token_index, reverse_unified_token_index)
    # # portuguese_translator = Translator(portuguese_transformer, unified_token_index, reverse_unified_token_index, unified_token_index, reverse_unified_token_index)
    # portuguese_translated_text = translator(
    #     sentence,15)
    # for i in portuguese_translated_text:
    #     try:
    #         print(reverse_unified_token_index[i], end=' ')
    #     except Exception:
    #         print(i)
    #         continue
        #transformer.save_weights(f'./models/spanish-portuguese_full_epoch{t}.weights.h5')

28510 28510 28510
Epoch 0:


/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer doe

891/891 ━━━━━━━━━━━━━━━━━━━━ 634s 709ms/step - loss: 7.8491 - masked_accuracy: 0.6261


/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (1, 8, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


0.10454156775170706
['[POSTART]', 'E', 'o', 'SENHOR', ',', 'e', 'o', 'SENHOR', ',', 'e', 'o', 'SENHOR', ',', 'e', 'o', 'SENHOR', ',', 'e', 'o', 'SENHOR', '.']
Epoch 1:
891/891 ━━━━━━━━━━━━━━━━━━━━ 644s 721ms/step - loss: 1.2354 - masked_accuracy: 0.8129
0.28154190270406404
['[POSTART]', 'O', 'anjo', 'no', 'dia', 'dos', 'céus', ',', 'e', 'os', 'céus', ',', 'e', 'a', 'terra', '.', '[END]']
Epoch 2:
891/891 ━━━━━━━━━━━━━━━━━━━━ 638s 716ms/step - loss: 0.9161 - masked_accuracy: 0.8532
0.28364881580741746
['[POSTART]', 'No', 'princípio', 'do', 'princípio', 'do', 'reino', 'dos', 'céus', 'e', 'a', 'terra', '.', '[END]']
Epoch 3:
891/891 ━━━━━━━━━━━━━━━━━━━━ 637s 715ms/step - loss: 0.7423 - masked_accuracy: 0.8745
0.43708845359073206
['[POSTART]', 'No', 'princípio', 'do', 'SENHOR', 'teu', 'deus', 'nos', 'céus', 'e', 'a', 'terra', '.', '[END]']
Epoch 4:
891/891 ━━━━━━━━━━━━━━━━━━━━ 637s 715ms/step - loss: 0.6419 - masked_accuracy: 0.8860
0.49426857709562527
['[POSTART]', 'No', 'ano', 'criou', '

In [50]:
tf.saved_model.save(transformer, export_dir=f'./translators/{t}')
transformer = tf.saved_model.load(export_dir='./translators/0')

INFO:tensorflow:Assets written to: ./translators/1/assets


INFO:tensorflow:Assets written to: ./translators/1/assets


In [28]:
# spanish_transformer.save_weights('./models/spanish120epoch.weights.h5')
# portuguese_transformer.save_weights('./models/portuguese120epoch.weights.h5')
#transformer.save_weights('./models/spanish-portuguese_full_epoch59.weights.h5')
# transformer.save('./models/spanish-portuguese_full_test.keras') # https://discuss.tensorflow.org/t/save-a-tensorflow-model-with-a-transformer-layer/7221
# transformer = tf.keras.models.load_model('./models/spanish-portuguese_full_test.keras') # https://discuss.tensorflow.org/t/save-a-tensorflow-model-with-a-transformer-layer/7221
transformer.load_weights('./models/spanish-portuguese_full_epoch9.weights.h5')

/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 1 variables whereas the saved optimizer has 345 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [51]:
# Define a translator
class Translator(tf.Module):
  def __init__(self, transformer, input_token_index, reverse_input_token_index, output_token_index, reverse_output_token_index):
    self.transformer = transformer
    self.input_token_index = input_token_index
    self.output_token_index = output_token_index
    self.reverse_input_token_index = reverse_input_token_index
    self.reverse_output_token_index = reverse_output_token_index

  def __call__(self, sentence, max_length):
    # The input sentence is Portuguese, hence adding the `[START]` and `[END]` tokens.
    # assert isinstance(sentence, tf.Tensor)
    # if len(sentence.shape) == 0:
    #   sentence = sentence[tf.newaxis]
    tokenized_sentence = ['[SPSTART]'] + nltk.tokenize.casual_tokenize(sentence) + ['[END]']
    sentence_tensor = np.zeros(max_length, dtype='int64')
    for t, token in enumerate(tokenized_sentence):
      sentence_tensor[t] = self.input_token_index[token]
    #sentence = self.tokenizers.pt.tokenize(sentence).to_tensor()

    encoder_input = sentence_tensor[tf.newaxis]
    
    # As the output language is English, initialize the output with the
    # English `[START]` token.
    #start_end = self.tokenizers.en.tokenize([''])[0]
    start = self.input_token_index['[POSTART]']
    end = self.output_token_index['[END]']

    # `tf.TensorArray` is required here (instead of a Python list), so that the
    # dynamic-loop can be traced by `tf.function`.
    output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
    output_array = output_array.write(0, start)
    for i in tf.range(max_length):
      output = tf.transpose(output_array.stack())[tf.newaxis]
      predictions = self.transformer([encoder_input, output], training=False)

      # Select the last token from the `seq_len` dimension.
      predictions = predictions[:, -1:, :]  # Shape `(batch_size, 1, vocab_size)`.

      predicted_id = tf.argmax(predictions, axis=-1)

      # Concatenate the `predicted_id` to the output which is given to the
      # decoder as its input.
      output_array = output_array.write(i+1, predicted_id[0][0])

      if predicted_id == end:
        break

    output = tf.transpose(output_array.stack())
    # The output shape is `(1, tokens)`. https://www.tensorflow.org/api_docs/python/tf/cast
    text = [item.numpy() for item in output]  # Shape: `()`.

    # `tf.function` prevents us from using the attention_weights that were
    # calculated on the last iteration of the loop.
    # So, recalculate them outside the loop.
    # self.transformer([encoder_input, output[:,:-1]], training=False)
    # attention_weights = self.transformer.decoder.last_attn_scores

    return text

In [52]:
# Test the translator
sentence = 'Estas son las palabras de Amós, que era un pastor de Tecoa'
translator = Translator(transformer, unified_token_index, reverse_unified_token_index, unified_token_index, reverse_unified_token_index)
# portuguese_translator = Translator(portuguese_transformer, unified_token_index, reverse_unified_token_index, unified_token_index, reverse_unified_token_index)
portuguese_translated_text = translator(
    sentence,15)
# portuguese_translated_text = portuguese_translator(
#     sentence,15)

TypeError: '_UserObject' object is not callable

In [31]:
# Print the raw lists
#print(spanish_translated_text)
print(portuguese_translated_text)

[43282, 26068, 21446, 8956, 2923, 1776, 44249, 33554, 44249, 14713, 13664, 19175, 40547, 48550, 16274, 19832]


In [32]:
# Print the sentences themselves
# for i in range(20):
#     try:
#         print(reverse_spanish_token_index[spanish_translated_text[i]], end=' ')
#     except Exception:
#         continue
# print()
for i in portuguese_translated_text:
    try:
        print(reverse_unified_token_index[i], end=' ')
    except Exception:
        print(i)
        continue

[POSTART] progresarán hendidura envidies composto ventre exacta intentaram exacta hendiste revés sereditas zelo secou Maldade - 

In [43]:
del translator
gc.collect()

2867